<a href="https://colab.research.google.com/github/armanheydari/Advance-Deep-Learning_Winter-2024/blob/main/Assignment2/cmpt489_828_a2_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CMPT 489/828 Assignment 2**

Follow the instructions in this notebook and complete the missing code.

**NOTE: Do Not Change Any Provided Code or Given Variable Names! Except changing gpu.**

**Q2**. Create a simple convolutional neural network using **torch.nn** module (**20 points**)


In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
import torchvision
from torchvision import transforms

# select gpu if possible
# you can change "cuda:0" to select other gpus
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# load CIFAR-10 dataset with pytorch
# set batch_size
batch_size = 100
# convert to tensor, normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_id = list(range(4000))
val_id = list(range(4000, 5000))
test_id = list(range(500))

# subset dataset and create dataloader with batch_size
train_sub_set = torch.utils.data.Subset(trainset, train_id)
val_sub_set = torch.utils.data.Subset(trainset, val_id)
test_sub_set = torch.utils.data.Subset(testset, test_id)

train_loader = torch.utils.data.DataLoader(train_sub_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_sub_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_sub_set, batch_size=batch_size, shuffle=True)

# check data size, should be (C,H,W), class map only useful for visualization and sanity checks
image_size = trainset[0][0].size()
class_map = {0: 'plane', 1: 'car', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship',
             9: 'truck'}

a. Implement a CNN model (**4 points**)

In [ ]:
class SimpleCnn(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        ###############################################################################
        # TODO:                                                                       #
        # 1. create conv1 layer:                                                      #
        #   with 32 output channels, 5x5 kernels, use default for others              #
        # 2. create conv2 layer:                                                      #
        #   with 64 output channels, 5x5 kernels, use default for others              #
        # 3. create linear layer fc1: with nb_hidden output channels                  #
        # 4. create linear layer fc2: with 10 output channels                         #
        ###############################################################################
        # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        self.conv_layer1 =
        self.conv_layer2 =
        self.fully_connected1 =
        self.fully_connected2 =
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    def forward(self, x):
        """
        forward step
        :param x: input tensor
        :return: output tensor
        """
        ###############################################################################
        # TODO:                                                                       #
        # 1. create first convolution block c1_out:                                   #
        #   relu(max_pool(conv1(), kernel_size=3, stride=3)                           #
        # 2. create second convolution block c2_out:                                  #
        #   relu(max_pool(conv2(), kernel_size=2, stride=2)                           #
        # 3. create fully connected block fc1_out: relu(fc1())                        #
        # 4. connect last fully connected layer fc2_out: fc2()                        #
        # 5. return fc2_out                                                           #
        ###############################################################################
        # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        c1_out =
        c2_out =
        fc1_out =
        fc2_out =
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

b. Train function (**8 points**)

In [ ]:
# train function
def train_model(model, train_loader, val_loader, nb_epochs=100):
    ###############################################################################
    # TODO:                                                                       #
    # 1. create loss: criterion use CrossEntropyLoss()                            #
    # 2. create optimizer: optimizer use optim.Adam()                             #
    ###############################################################################
    # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    criterion =
    optimizer =
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # initialize loss/acc dict storage
    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }
    train_steps = len(train_loader.dataset) // batch_size
    val_steps = len(val_loader.dataset) // batch_size

    # run for nb_epochs
    for e in range(nb_epochs):
        # set the model in training mode
        model.train()
        # initialize the total training and validation loss
        epoch_train_loss = 0
        epoch_val_loss = 0
        # initialize the number of correct predictions in the training
        # and validation step
        train_correct = 0
        val_correct = 0

        for x, y in train_loader:
            ###############################################################################
            # TODO:                                                                       #
            # 1. move x, y to device                                                      #
            # 2. clear optimizer gradients                                                #
            # 3. predict batch x, save in pred                                            #
            # 4. calculate batch loss, save in loss                                       #
            # 5. step optimizer                                                           #
            ###############################################################################
            # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            # add the loss to the total training loss so far and
            # calculate the number of correct predictions
            epoch_train_loss += loss
            train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # switch off autograd for validation
        with torch.no_grad():
            # set the model in evaluation mode
            model.eval()
            # loop over the validation set
            for (x, y) in val_loader:
                ###############################################################################
                # TODO:                                                                       #
                # 1. move x, y to device                                                      #
                # 2. predict batch x, save in pred                                            #
                # 3. update epoch_val_loss                                                    #
                # 5. update val_correct                                                       #
                ###############################################################################
                # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

                # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


        # calculate the average epoch training and validation loss
        mean_train_loss = epoch_train_loss / train_steps
        mean_val_loss = epoch_val_loss / val_steps
        # calculate the training and validation accuracy
        train_correct = train_correct / len(train_loader.dataset)
        val_correct = val_correct / len(val_loader.dataset)
        # update our training history
        history["train_loss"].append(mean_train_loss.cpu().detach().numpy())
        history["train_acc"].append(train_correct)
        history["val_loss"].append(mean_val_loss.cpu().detach().numpy())
        history["val_acc"].append(val_correct)
        # print the model training and validation information
        print("[INFO] EPOCH: {}/{}".format(e + 1, nb_epochs))
        print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
            mean_train_loss, train_correct))
        print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
            mean_val_loss, val_correct))

c. Test function (**3 points**)

In [ ]:
def test(model, test_loader):
    # we can now evaluate the network on the test set
    print("[INFO] testing SimpleCnn...")
    # turn off autograd for testing evaluation
    ###############################################################################
    # TODO:                                                                       #
    # 1. initialize test correct counter: test_correct                            #
    # 2. switch off autograd                                                      #
    # 3. put model in evaluation mode                                             #
    # 4. loop over test_loader                                                    #
    # 5. move data to device                                                      #
    # 6. predict batch x, save in pred                                            #
    # 7. update test_correct                                                      #
    # 8. calculate average test accuracy                                          #
    # 9. print average test accuracy                                              #
    ###############################################################################
    # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

d. Train and test your model (**3 points**)

In [ ]:
###############################################################################
# TODO:                                                                       #
# 1. create model instance with 500 neurons for first linear layer            #
# 2. call training loop, train for 300 epochs                                 #
# 3. call test function                                                       #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
model =
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

d. Experiment with model architecture (**2 points**)

Try different number of neurons for the first linear layer. Do you notice any performance difference?

Also try a CNN with an additional conv block (conv+pool+relu).

You can use different kernerl sizes for you conv/pool layer.


Do you notice any performance difference?

In [ ]:
###############################################################################
# TODO:                                                                       #
# 1. Experiment with different nb_hidden values for your CNN                  #
# Take note of the performance changes                                        #
# 2. Create another CNN with one more conv block                              #
# Take note of the performance changes                                        #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

In [ ]:
import gc
# clean up, release memory
model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()